In [1]:
# -*-coding:utf-8-*-
# Edited by bighead 19-2-15

import numpy as np
import pandas as pd
import cv2
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
##### get hog feature
def get_hot_features(trainset):
    features=[]
    hog = cv2.HOGDescriptor('../data/hog.xml')
    
    for img in trainset:
        img = np.reshape(img, (28,28))
        cv_img = img.astype(np.uint8)
        
        hog_feature = hog.compute(cv_img)
        features.append(hog_feature)
        
    features = np.array(features)
    features = np.reshape(features, (-1, 324))
    
    return features

In [3]:
##### define the neighbors k
k = 10

In [4]:
##### K nearest neightbors model using KDTree
KNC = KNeighborsClassifier(n_neighbors=k, weights='uniform', algorithm='kd_tree')

In [5]:
##### main
print("Start read the data")
time_1 = time.time()

raw_data = pd.read_csv('../data/train.csv', header=0)
data = raw_data.values

imgs=data[:, 1:]
labels=data[:,0]

features=get_hot_features(imgs)

### train test set split
features_train, features_test, label_train, label_test = train_test_split(features, labels, test_size=0.33, random_state=23323)

time_2 = time.time()
print("read data finish! cost: {} seconds".format(time_2-time_1))

### training
print("start training")
print("fit the sklearn knn model")
KNC.fit(features, labels)
time_3 = time.time()
print("training cost: {} seconds".format(time_3-time_2))

### predicting
print("start predictiing")
label_predict = KNC.predict(features_test)
time_4 = time.time()
print("predicting cost: {} seconds".format(time_4-time_3))

### score
score = accuracy_score(label_test, label_predict)
print("The accruacy scores is {}".format(score))

Start read the data
read data finish! cost: 6.01065802574 seconds
start training
fit the sklearn knn model
training cost: 1.02905297279 seconds
start predictiing
predicting cost: 375.218507051 seconds
The accruacy scores is 0.987301587302


In [6]:
# print features_train.shape

In [7]:
# print label_train.shape

In [8]:
### load test data
kaggle_test_features = pd.read_csv('../data/test.csv').values
kaggle_test_features = get_hot_features(kaggle_test_features)

In [9]:
### kaggle mnist predict
time_start = time.time()
kaggle_predict = KNC.predict(kaggle_test_features)
print("finish predicting kaggle test data, cost: {}".format(time.time()-time_start))

finish predicting kaggle test data, cost: 756.729955912


In [10]:
kaggle_predict
print(kaggle_predict.shape)
submit = pd.DataFrame(columns=['ImageId', 'Label'])
submit['ImageId'] = [i for i in range(1, len(kaggle_predict)+1)]
submit['Label'] = kaggle_predict

(28000,)


In [11]:
submit.to_csv("submit_knn.csv", index=False)